# Libreta 10 - Uso de SQL y LLMs para creación de recomendaciones Parte 1

**Profesor:**  Dr. Farid Krayem Pineda

**Materia:**  Minería de Datos

**Universidad Anáhuac Puebla**

---

In [ ]:
# Instalamos PyMySQL para compatibilidad entre Python y SQL

!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
# Instalación de librerías

import pandas as pd
import xml.etree.ElementTree as ET
import requests
from google import genai
from google.colab import userdata
import time
import pymysql
import random

In [ ]:
# Aquí usa tu API KEY de Gemini

api_key_gemini = userdata.get('api_key_gemini_gmail')

In [ ]:
## Creamos un dataframe a partir del sitemap de productos

# URL del sitemap
sitemap_url = "https://fredzapaterias.com.mx/sitemap_products_1.xml?from=4604151169112&to=9125506482405"

# Descargar el sitemap
response = requests.get(sitemap_url)
response.raise_for_status()

# Parsear el contenido
root = ET.fromstring(response.content)

# Definir namespaces
namespaces = {
    "ns": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "image": "http://www.google.com/schemas/sitemap-image/1.1"
}

# Crear listas para guardar datos
productos = []
imagenes = []

# Extraer información
for url in root.findall("ns:url", namespaces):
    image_title = url.find("image:image/image:title", namespaces)
    image_loc = url.find("image:image/image:loc", namespaces)

    if image_title is not None and image_loc is not None:
        productos.append(image_title.text)
        imagenes.append(image_loc.text)

# Crear el DataFrame
programs_df = pd.DataFrame({
    "Producto": productos,
    "Imagen": imagenes
})

programs_df


,Producto,Imagen
0,Sandalias de Piso para Hombre Caray FOREVER,https://cdn.shopify.com/s/files/1/0285/8585/30...
1,Zapatos Hombre Sandalia De Piso Para Playa Car...,https://cdn.shopify.com/s/files/1/0285/8585/30...
2,Zapatos Bebés Bota De Niña Con Hebilla Sandy 5062,https://cdn.shopify.com/s/files/1/0285/8585/30...
3,Zapatos Bebés Bota De Niña Casual Con Hebilla ...,https://cdn.shopify.com/s/files/1/0285/8585/30...
4,Zapatos Bebés Bota De Niño Casual Con Agujetas...,https://cdn.shopify.com/s/files/1/0285/8585/30...
...,...,...
1961,Tenis Casuales con Agujetas para Hombre Puma 3...,https://cdn.shopify.com/s/files/1/0285/8585/30...
1962,Tenis Casuales con Agujetas para Hombre Puma 3...,https://cdn.shopify.com/s/files/1/0285/8585/30...
1963,Tenis Deportivos con Agujetas para Hombre Reeb...,https://cdn.shopify.com/s/files/1/0285/8585/30...
1964,Tenis Casuales con Agujetas para Hombre Skeche...,https://cdn.shopify.com/s/files/1/0285/8585/30...


In [ ]:
## Guardamos los 15 primeros resultados de los productos ##

prod_fin_df = programs_df.head(15)
prod_fin_df


,Producto,Imagen
0,Sandalias de Piso para Hombre Caray FOREVER,https://cdn.shopify.com/s/files/1/0285/8585/30...
1,Zapatos Hombre Sandalia De Piso Para Playa Car...,https://cdn.shopify.com/s/files/1/0285/8585/30...
2,Zapatos Bebés Bota De Niña Con Hebilla Sandy 5062,https://cdn.shopify.com/s/files/1/0285/8585/30...
3,Zapatos Bebés Bota De Niña Casual Con Hebilla ...,https://cdn.shopify.com/s/files/1/0285/8585/30...
4,Zapatos Bebés Bota De Niño Casual Con Agujetas...,https://cdn.shopify.com/s/files/1/0285/8585/30...
5,Zapatos Bebés Bota De Niño Casual Con Velcro S...,https://cdn.shopify.com/s/files/1/0285/8585/30...
6,Botas Industriales con Casquillo para Hombre R...,https://cdn.shopify.com/s/files/1/0285/8585/30...
7,Botas Vaqueras Para Hombre Rio Grande 291,https://cdn.shopify.com/s/files/1/0285/8585/30...
8,Bota Industrial Con Agujetas Y Casquillo para ...,https://cdn.shopify.com/s/files/1/0285/8585/30...
9,Zapatos Hombre De Vestir Con Agujetas Quirelli...,https://cdn.shopify.com/s/files/1/0285/8585/30...


In [ ]:
client = genai.Client(api_key=api_key_gemini)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Crea un haiku sobre minería de datos. ",
)

print(response.text)

Datos desenterrados,
Conocimiento se encuentra brillante,
Patrones revelados.



In [ ]:
def respuesta_chat(rol,prompt):
  response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=rol + " " + prompt,
  )

  respuesta = response.text

  return respuesta

In [ ]:
respuesta_chat("Eres un poeta mexicano","Tu tarea es hacer un poema de 20 palabras sobre los mayas.")

'Claro, aquí te va:\n\nEn selva verde, pirámides de piedra,\nsabios mayas, estrellas lejanas,\ncalendario eterno, un eco ancestral.\n'

In [ ]:
## Usamos Google Gemini para generar las descripciones del producto

# Inicializar cliente
client = genai.Client(api_key=api_key_gemini)

# Crear lista para las descripciones
descripciones = []

# Generar descripciones con control de rate limit
for idx, producto in enumerate(prod_fin_df["Producto"]):
    if idx > 0 and idx % 14 == 0:
        time.sleep(60)  # Esperar 60 segundos después de 14 solicitudes

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"Genera una descripción de producto de aproximadamente 70 palabras sobre el producto: {producto}. El tono debe ser claro y persuasivo."
    )
    descripciones.append(response.text)

# Agregar la columna al DataFrame
prod_fin_df["Descripción"] = descripciones

prod_fin_df


<ipython-input-9-a3f0c7caf994>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_fin_df["Descripción"] = descripciones


,Producto,Imagen,Descripción
0,Sandalias de Piso para Hombre Caray FOREVER,https://cdn.shopify.com/s/files/1/0285/8585/30...,Disfruta la comodidad y el estilo atemporal co...
1,Zapatos Hombre Sandalia De Piso Para Playa Car...,https://cdn.shopify.com/s/files/1/0285/8585/30...,Prepárate para la playa con las sandalias Cara...
2,Zapatos Bebés Bota De Niña Con Hebilla Sandy 5062,https://cdn.shopify.com/s/files/1/0285/8585/30...,¡Prepárate para adorar las Botas Sandy 5062! E...
3,Zapatos Bebés Bota De Niña Casual Con Hebilla ...,https://cdn.shopify.com/s/files/1/0285/8585/30...,¡Prepárate para robar corazones con las botas ...
4,Zapatos Bebés Bota De Niño Casual Con Agujetas...,https://cdn.shopify.com/s/files/1/0285/8585/30...,¡Dale a tu pequeño aventurero sus primeros pas...
5,Zapatos Bebés Bota De Niño Casual Con Velcro S...,https://cdn.shopify.com/s/files/1/0285/8585/30...,¡Prepárate para la aventura con las Botas Sand...
6,Botas Industriales con Casquillo para Hombre R...,https://cdn.shopify.com/s/files/1/0285/8585/30...,Domina cualquier terreno con las Botas Industr...
7,Botas Vaqueras Para Hombre Rio Grande 291,https://cdn.shopify.com/s/files/1/0285/8585/30...,Domina el estilo vaquero con las Botas Rio Gra...
8,Bota Industrial Con Agujetas Y Casquillo para ...,https://cdn.shopify.com/s/files/1/0285/8585/30...,Domina el trabajo duro con la Bota Industrial ...
9,Zapatos Hombre De Vestir Con Agujetas Quirelli...,https://cdn.shopify.com/s/files/1/0285/8585/30...,Eleva tu estilo con los zapatos de vestir Quir...


In [ ]:
## Impresión de un registro para validación

# Seleccionar un índice aleatorio
random_index = random.randint(0, len(prod_fin_df) - 1)

# Obtener la descripción correspondiente
descripcion_prueb = prod_fin_df["Descripción"].iloc[random_index]

# Imprimir resultados con mejor formato
print("="*50)
print(f"📄 Registro seleccionado: {random_index}")
print(f"📝 Número de palabras: {len(descripcion_prueb.split())}")
print("\nDescripción:\n")
print(descripcion_prueb)
print("="*50)

📄 Registro seleccionado: 2
📝 Número de palabras: 78

Descripción:

¡Prepárate para adorar las Botas Sandy 5062! Estas adorables botas para bebé niña combinan estilo y comodidad a la perfección. Fabricadas con materiales suaves y duraderos, cuentan con un cierre de hebilla que asegura un ajuste perfecto para esos piecitos en crecimiento. Su diseño clásico y femenino las hace ideales para cualquier ocasión, desde paseos en el parque hasta fiestas especiales. ¡Dale a tu pequeña el estilo y la comodidad que se merece con las Botas Sandy 5062!



In [ ]:
## Probar la conexión con la base de datos.

pass_db = userdata.get('pass_db_anahua')

# Configura los detalles de conexión a la base de datos
host = "35.215.110.77"
puerto = 3306
usuario = "ugu1kvcishvbx"
contraseña = pass_db
base_de_datos = "dboewuclpuqgcr"

# Intenta conectarte a la base de datos
try:
    conexion = pymysql.connect(
        host=host,
        user=usuario,
        password=contraseña,
        database=base_de_datos
    )
    print("Conexión exitosa a la base de datos.")

    # No olvides cerrar la conexión cuando hayas terminado
    conexion.close()

except pymysql.Error as e:
    print("Error al conectar a la base de datos:", e)


Conexión exitosa a la base de datos.


In [ ]:
# Configura los detalles de conexión a la base de datos
host = "35.215.110.77"
puerto = 3306
usuario = "ugu1kvcishvbx"
contraseña = pass_db
base_de_datos = "dboewuclpuqgcr"

# Conectarse a la base de datos
try:
    conexion = pymysql.connect(
        host=host,
        user=usuario,
        password=contraseña,
        database=base_de_datos
    )
    cursor = conexion.cursor()

    # Verificar si existe la tabla
    cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = %s
        AND table_name = 'prueba_productos'
    """, (base_de_datos,))

    if cursor.fetchone()[0] == 1:
        cursor.execute("DROP TABLE prueba_productos")
        print("Tabla 'prueba_productos' eliminada.")

    # Crear la tabla
    cursor.execute("""
        CREATE TABLE prueba_productos (
            id INT AUTO_INCREMENT PRIMARY KEY,
            Producto TEXT,
            Imagen TEXT,
            Descripcion TEXT
        )
    """)
    print("Tabla 'prueba_productos' creada.")

    # Insertar datos de prod_fin_df
    for _, row in prod_fin_df.iterrows():
        cursor.execute("""
            INSERT INTO prueba_productos (Producto, Imagen, Descripcion)
            VALUES (%s, %s, %s)
        """, (row["Producto"], row["Imagen"], row.get("Descripción", "")))

    conexion.commit()
    print("Datos insertados correctamente.")

    conexion.close()

except pymysql.Error as e:
    print("Error al conectar o manipular la base de datos:", e)

Tabla 'prueba_productos' eliminada.
Tabla 'prueba_productos' creada.
Datos insertados correctamente.


In [ ]:
# Conectarse y consultar
try:
    conexion = pymysql.connect(
        host=host,
        user=usuario,
        password=contraseña,
        database=base_de_datos
    )
    cursor = conexion.cursor()

    # Consultar todos los registros
    cursor.execute("SELECT * FROM prueba_productos")
    resultados = cursor.fetchall()

    for fila in resultados:
        print(fila)

    conexion.close()

except pymysql.Error as e:
    print("Error al conectar o consultar la base de datos:", e)

(1, 'Sandalias de Piso para Hombre Caray FOREVER', 'https://cdn.shopify.com/s/files/1/0285/8585/3016/products/zapatos-hombre-sandalias-caray-forever-cafe-lateral.jpg?v=1741298765', 'Disfruta la comodidad y el estilo atemporal con las Sandalias de Piso para Hombre Caray FOREVER. Fabricadas con materiales duraderos y un diseño ergonómico, estas sandalias te brindan soporte y frescura durante todo el día. Ideales para la playa, la piscina o simplemente relajarte en casa, las FOREVER son la opción perfecta para el hombre moderno que busca confort sin sacrificar la elegancia. ¡Pisa con confianza y estilo con Caray FOREVER!\n')
(2, 'Zapatos Hombre Sandalia De Piso Para Playa Caray Georgia', 'https://cdn.shopify.com/s/files/1/0285/8585/3016/products/caray-georgia-cafe.jpg?v=1591168722', 'Prepárate para la playa con las sandalias Caray Georgia. Diseñadas para la comodidad y el estilo, estas sandalias de piso para hombre son perfectas para tus aventuras veraniegas. Fabricadas con materiales dur

In [ ]:
# Configura los detalles de conexión a la base de datos
host = "35.215.110.77"
puerto = 3306
usuario = "ugu1kvcishvbx"
contraseña = pass_db
base_de_datos = "dboewuclpuqgcr"

# Conectarse y consultar columnas
try:
    conexion = pymysql.connect(
        host=host,
        user=usuario,
        password=contraseña,
        database=base_de_datos
    )
    cursor = conexion.cursor()

    # Consultar solo las columnas de la tabla
    cursor.execute("""
        SELECT COLUMN_NAME
        FROM information_schema.columns
        WHERE table_schema = %s
        AND table_name = 'prueba_productos'
    """, (base_de_datos,))

    columnas = cursor.fetchall()

    for columna in columnas:
        print(columna[0])

    conexion.close()

except pymysql.Error as e:
    print("Error al conectar o consultar columnas de la base de datos:", e)


id
Producto
Imagen
Descripcion


In [ ]:
# Vamos a meter un valor de forma manual

# Configura los detalles de conexión a la base de datos
host = "35.215.110.77"
puerto = 3306
usuario = "ugu1kvcishvbx"
contraseña = pass_db
base_de_datos = "dboewuclpuqgcr"

# Datos a insertar manualmente
producto = "Bicicleta de Montaña Minería de Datos"
imagen = "https://elektra.vtexassets.com/arquivos/ids/9531890-1200-auto?v=638700586835800000&width=1200&height=auto&aspect=true"
descripcion = "Experimenta la aventura con la Bicicleta de Minería de Datos. Diseñada para conquistar terrenos desafiantes, su cuadro resistente y suspensión delantera absorben los impactos, brindándote comodidad y control. Sus 21 velocidades te permiten adaptarte a cualquier pendiente, mientras que sus frenos de disco te garantizan seguridad en cada descenso. ¡Libera tu espíritu aventurero y explora nuevos horizontes con la Ranger R27.5!"

# Conectarse y agregar registro
try:
    conexion = pymysql.connect(
        host=host,
        user=usuario,
        password=contraseña,
        database=base_de_datos
    )
    cursor = conexion.cursor()

    cursor.execute("""
        INSERT INTO prueba_productos (Producto, Imagen, Descripcion)
        VALUES (%s, %s, %s)
    """, (producto, imagen, descripcion))

    conexion.commit()
    print("Registro agregado correctamente.")

    conexion.close()

except pymysql.Error as e:
    print("Error al conectar o insertar el registro:", e)


Registro agregado correctamente.


In [ ]:
# Configura los detalles de conexión a la base de datos
host = "35.215.110.77"
puerto = 3306
usuario = "ugu1kvcishvbx"
contraseña = pass_db
base_de_datos = "dboewuclpuqgcr"

# Conectarse y eliminar el producto
try:
    conexion = pymysql.connect(
        host=host,
        user=usuario,
        password=contraseña,
        database=base_de_datos
    )
    cursor = conexion.cursor()

    # Eliminar el producto específico
    cursor.execute("""
        DELETE FROM prueba_productos
        WHERE Producto = %s
    """, ("Bicicleta de Montaña Ranger R27.5 21V Gris",))

    conexion.commit()
    print("Producto eliminado correctamente.")

    conexion.close()

except pymysql.Error as e:
    print("Error al conectar o eliminar el producto:", e)


Producto eliminado correctamente.


## Revisa tu resultado en:

[Anáhuac Moda](https://mercabits.com/dev/anahuac-moda/)